In [1]:
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
pd.options.display.max_columns = 200
pd.options.display.max_rows = 200
pd.options.display.max_colwidth = 5000

In [3]:
from IPython.display import display, HTML

def pretty_print(df):
    return display( HTML( df.to_html().replace("\\n","<br>") ) )

## Profile Questions

- Service Type?
    - CAF 
    - RCMP 
    - War Service Veteran

- Patron Type?
    - Service Member
    - Family

- Is Still Serving?
    - None if Service Type != CAF
    - Is Still Serving
    - Released
 
- Veteran Is Alive?
    - None if Patron Type == Service Member
    - Veteran Is Alive
    - Veteran is Deceased   

# Get Data

In [4]:
df_results_orig = pd.read_excel('../data/VAC Data.xlsx', sheet_name=3)
df_mapping_orig = pd.read_excel('../data/VAC Data.xlsx', sheet_name=2)

In [5]:
df_mapping = df_mapping_orig.rename(columns={df_mapping_orig.columns[0]: "Result ID", 
                                         "Statements": "Scenario",
                                         "Unnamed: 129": "Service Type",
                                         "Unnamed: 127": "Result Name",
                                         "Unnamed: 130": "Applicant",
                                         "Unnamed: 131": "Benefit",
                                         "Unnamed: 132": "Additional Details",
                                         "Unnamed: 128": "Result Description",
                                        })
df_mapping = df_mapping.drop(df_mapping.index[0])

## Member or Veteran died as a result of a service related injury or illness 

In [6]:
df = df_mapping[['Benefit', 'still serving', 'Member or Veteran died as a result of a service related injury or illness']]
df.drop_duplicates().head()

,Benefit,still serving,Member or Veteran died as a result of a service related injury or illness
11,Disability Benefits,NaN,NaN
61,Family Caregiver Relief Benefit,NaN,NaN
71,Attendance Allowance,NaN,NaN
81,Clothing Allowance,NaN,NaN
91,Exceptional Incapacity Allowance,Y,NaN


## Family benefits: Veteran deceased vs alive

In [7]:
df = df_mapping[['Benefit', 'Applicant']].drop_duplicates()
df = df[df['Applicant'] != 'Member']
        
df['Veteran Alive'] = df.Applicant.map(lambda s: 'Deceased' if s in ['Survivor', 'Orphan (Veteran is deceased)'] else 'Alive')
df = df[['Benefit', 'Veteran Alive']].drop_duplicates()
df.sort_values(by=['Veteran Alive', 'Benefit']).head()

,Benefit,Veteran Alive
100161,Attendance Allowance,Alive
100141,Disability Benefits,Alive
100171,Family Caregiver Relief Benefit,Alive
100363,Memorial Ribbon,Alive
100381,OSISS Family Support,Alive


## All together

In [8]:
df = df_mapping[['Benefit', 'Service Type', 'Applicant', 'still serving']].drop_duplicates()

def still_serving_munger(x):
    if x.lower() == 'y':
        return 'Still Serving'
    elif x.lower() == 'n':
        return 'Released'
    else:
        return x
df['Is Still Serving'] = df['still serving']. fillna('Unknown').map(still_serving_munger)

df['Patron Type'] = df['Applicant'].map(lambda s: 'Member' if s == 'Member' else 'Family')

df['Is Veteran Alive'] = df.Applicant.map(lambda s: 'Veteran is Deceased' if s in ['Survivor', 'Orphan (Veteran is deceased)'] else 'Veteran is Alive')

df = df[['Benefit', 'Service Type', 'Patron Type', 'Is Still Serving', 'Is Veteran Alive']].drop_duplicates()
df.sort_values(by="Benefit", inplace=True)
df = df.reset_index(drop=True)
df.head()

,Benefit,Service Type,Patron Type,Is Still Serving,Is Veteran Alive
0,Attendance Allowance,War Service Veteran,Family,Unknown,Veteran is Alive
1,Attendance Allowance,CAF,Member,Unknown,Veteran is Alive
2,Attendance Allowance,War Service Veteran,Member,Unknown,Veteran is Alive
3,Attendance Allowance,RCMP,Member,Unknown,Veteran is Alive
4,Attendance Allowance,Allied Veteran,Member,Unknown,Veteran is Alive


In [9]:
df.to_csv('profile_data.csv')

## Reserve status

In [10]:
reserve_df = df_mapping[["Benefit", 'member or Veteran of the Regular Force', 'Reserve force Veteran or member  completed at least 21 months of full time service during 24 consecutive months or Special Duty Service or emergency service within the last two years']]

In [11]:
reserve_df = reserve_df.drop_duplicates()

In [12]:
reserve_df[reserve_df['member or Veteran of the Regular Force'].notnull() 
           | reserve_df['Reserve force Veteran or member  completed at least 21 months of full time service during 24 consecutive months or Special Duty Service or emergency service within the last two years'].notnull()]

,Benefit,member or Veteran of the Regular Force,Reserve force Veteran or member completed at least 21 months of full time service during 24 consecutive months or Special Duty Service or emergency service within the last two years
221,Career Transition Services,Y,NaN
222,Career Transition Services,N,Y
